In [20]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import get_crypto_data as gcd
import pandas as pd
import matplotlib
import pyfolio as pf
from datetime import datetime

In [4]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [5]:
cerebro = bt.Cerebro()

In [6]:
binance = gcd.getBinanceExchange()
dataframe = gcd.get_DataFrame(['ETH/BTC'], binance, '1/1/18', '12/31/19')
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)

In [7]:
cerebro.addstrategy(MaCrossStrategy)

0

In [8]:
cerebro.broker.setcash(1000000.0)

In [9]:
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [10]:
cerebro.addanalyzer(bt.analyzers.PyFolio, _name = 'pyfolio')

In [11]:
strats = cerebro.run()

In [12]:
cerebro.broker.getvalue()

971104.7165789388

In [18]:
pyfoliozer = strats[0].analyzers.getbyname('pyfolio')

In [19]:
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

In [23]:
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    #gross_lev=gross_lev,
    #live_start_date='2005-05-01',  # This date is sample specific
    round_trips=True)

Start date,2018-01-01
End date,2019-12-31
Total months,34
,Backtest
Annual return,-1.0%
Cumulative returns,-2.9%
Annual volatility,2.1%
Sharpe ratio,-0.47
Calmar ratio,-0.18
Stability,0.61
Max drawdown,-5.5%


AttributeError: 'numpy.int64' object has no attribute 'to_pydatetime'